# Topics - a GloVe implementation


## TODO-list:
- make visualization of decisionprocess
- do topic for all words in the vocabulary

### Unrelated ideas
- We *could* do PCA to vizualize glove technology 

In [1]:
# external imports
import numpy as np
import pandas as pd
import seaborn as sb
import gensim as gs
import pylab as pl
import matplotlib.pyplot as plt
import csv
import scipy

# internal imports
import helpers as HP



# Constants: PS! put in your own paths to the files
GLOVE_FOLDER = 'glove.twitter.27B'
GS_FOLDER = 'gensim_glove_twitter_27B/'
GS_25DIM = GS_FOLDER + "gensim_glove_25dim.txt"
GS_50DIM = GS_FOLDER + "gensim_glove_50dim.txt"
GS_100DIM = GS_FOLDER + "gensim_glove_100dim.txt"
GS_200DIM = GS_FOLDER + "gensim_glove_200dim.txt"


## Gensim GloVe
one can use gensims word2vec functions to check similarity and other interesting functions
https://radimrehurek.com/gensim/models/word2vec.html

#### ONLY ONE TIME: create the gensim_files

In [2]:
# spits out a .txt-file with the vectors in gensim format
#HP.create_gensim_wv_from_glove(GLOVE_FOLDER)
# afterwards you can delete the originalglovefiles

#### Loading global vectors

In [3]:
# uses the created gensim-.txt file to create the word2vec so one can operate on it
# choose wordvectors with the dimension you want
global_vectors = HP.load_gensim_global_vectors(GS_50DIM)

#### Examples of use

In [4]:
global_vectors.similar_by_word("racism")

[('homophobia', 0.846714437007904),
 ('racial', 0.8199193477630615),
 ('sexism', 0.8197556734085083),
 ('misogyny', 0.8180930614471436),
 ('bigotry', 0.8014910221099854),
 ('discrimination', 0.7976340651512146),
 ('terrorism', 0.7949816584587097),
 ('racist', 0.7810259461402893),
 ('oppression', 0.7747340202331543),
 ('slavery', 0.7610248327255249)]

In [5]:
# supposed to show queen, to demonstrate the power of word2vec, but failed #blameTwitterDataset
global_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

[('stone', 0.9959613680839539),
 ('queen', 0.9848849177360535),
 ('royal', 0.9840918779373169),
 ('prince', 0.9837151765823364),
 ('meets', 0.981272280216217),
 ('african', 0.9724677205085754),
 ('virgin', 0.9554038643836975),
 ('american', 0.9529185891151428),
 ('elizabeth', 0.9502763152122498),
 ('baker', 0.9487670660018921)]

#### Generating related word to a topic
related words should maybe be choosen by hand, if not one can use this method. Define how many words you want to define the topic with topn=__

In [6]:
related_words = HP.generate_related_words("racism", global_vectors, topn=12)
print(related_words)

['homophobia', 'racial', 'sexism', 'misogyny', 'bigotry', 'discrimination', 'terrorism', 'racist', 'oppression', 'slavery', 'prejudice', 'hypocrisy']


#### Creating topic vector with associated standard deviations 

In [7]:
racism_topic_vector, racism_std_dims  = HP.create_topic(related_words, global_vectors)
print("racism_topic_vector:", racism_topic_vector)
print("Standard deviation per dimension:", racism_std_dims)

racism_topic_vector: [  5.3061676    4.14918756   2.2796092    4.47750092   3.64603496
   8.85480309   0.98863995  -2.06385827   2.20178533   0.07488099
   7.98649073  -9.27251053 -24.35789871   0.35689199   4.537889
  -7.00892019  -2.06071401  -4.60118055  10.23355961   0.72619623
  -1.88957298  -2.52386475   1.51116705   1.04792011   7.69085932
  14.34574986   4.22350407   8.24262047  -8.73881149   1.84188223
   5.30568695  -2.90989947  10.66682911  -8.85278893   1.79724383
  -4.21171045  -9.31853962   3.79521108  -1.17262411 -16.17009926
  10.79034042 -12.50316048   3.87884617  -9.67669106  -0.22767806
   4.65888023   1.32117307   4.59731483  -0.98539394 -10.02946854]
Standard deviation per dimension: [0.58725518, 0.29987451, 0.38871989, 0.37038887, 0.37891182, 0.3536672, 0.34381258, 0.26557761, 0.40413705, 0.29557183, 0.28638759, 0.27112234, 0.52783298, 0.58236593, 0.40554154, 0.42874056, 0.26720375, 0.45282122, 0.33696237, 0.41483811, 0.36824888, 0.38919759, 0.32197943, 0.33082637

#### Calculating a words relation to topic

In [17]:
# creating a wordvector
word_vector = global_vectors['nigger']
similarity_score = HP.calculate_topic_similarity(word_vector, racism_topic_vector, global_vectors, std_dims=False, perc_dim_to_compare=0.5)
print("niggers relation to racism", similarity_score)

word_vector = global_vectors['car']
similarity_score = HP.calculate_topic_similarity(word_vector, racism_topic_vector, global_vectors, std_dims=False, perc_dim_to_compare=0.5)
print("hope relation to racism", similarity_score)

niggers relation to racism 0.399370372295
hope relation to racism 0.165352985263


## Creating vizalization for decisionmaking of topics-words (+ for blog)

In [ ]:
def analyze_related_words(related_words):
    """ This function prints out the cosine distance between the wordvectors and the topicvector it creates from them.
    It is meant as a tool to analyse (numerical) outliers of the words you pick to represent the group so you are sure
    that you don't portrait the wrong meaning of the topic
    """
    # look up the words in the vectorspace
    word_vectors = [glove_model.wv[word] for word in related_words]
    # sum up the wordvectors to one big vector representing the average "meaning" of all the words
    topic_vector = sum(racism_vectors)
    # Compute cosine distance from the individual wordvectors: To identify 
    cosine_dist_to_topic= [(scipy.spatial.distance.cosine(topic_vector, vec)) for vec in word_vectors]
    print("Cosine distance from wordvector to topicvector:\n", dict(zip(related_words,cosine_dist_to_topic)))
    
    """ 
    TODO: plot overlapping polarcoordinates 
    """

analyze_related_words(racism)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.path as path
import matplotlib.pyplot as plt
import matplotlib.patches as patches



def polarplot_wordvector(wordvectors, yticks=None, ylim=None):
    
    # Choose some nice colors
    matplotlib.rc('axes', facecolor = 'white')

    # Make figure background the same colors as axes 
    fig = plt.figure(figsize=(10,8), facecolor='white')
    
    for i, wordvector in enumerate(wordvectors):
        # Data to be represented
        # ----------
        properties = range(len(wordvector))
        values = wordvector
        # ----------

        # Use a polar axes
        axes = plt.subplot(111, polar=True)

        # Set ticks to the number of properties (in radians)
        t = np.arange(0,2*np.pi,2*np.pi/len(properties))
        plt.xticks(t, [])

        # Set yticks from 0 to 10
        plt.yticks(np.linspace(np.min(wordvectors),np.max(wordvectors),5)) if yticks == None else plt.yticks(yticks)

        # Draw polygon representing values
        points = [(x,y) for x,y in zip(t,values)]
        points.append(points[0])
        points = np.array(points)
        codes = [path.Path.MOVETO,] + \
                [path.Path.LINETO,]*(len(values) -1) + \
                [ path.Path.CLOSEPOLY ]
        _path = path.Path(points, codes)
        _patch = patches.PathPatch(_path, fill=False, color='blue', linewidth=0, alpha=.1)
        axes.add_patch(_patch)
        _patch = patches.PathPatch(_path, fill=False, linewidth = 2)
        axes.add_patch(_patch)


        # Set axes limits
        plt.ylim(np.min(wordvectors),np.max(wordvectors)) if ylim == None else plt.ylim(ylim)

        # Draw ytick labels to make sure they fit properly
        for i in range(len(properties)):
            angle_rad = i/float(len(properties))*2*np.pi
            angle_deg = i/float(len(properties))*360
            ha = "right"
            if angle_rad < np.pi/2 or angle_rad > 3*np.pi/2: ha = "left"
            plt.text(angle_rad, np.max(values), properties[i], size=14,
                     horizontalalignment=ha, verticalalignment="center")

            # A variant on label orientation
            #    plt.text(angle_rad, 11, properties[i], size=14,
            #             rotation=angle_deg-90,
            #             horizontalalignment='center', verticalalignment="center")

        # Done
        plt.hold

    plt.show()

In [ ]:
polarplot_wordvector(wordvectors=racism_vectors[0:2])

## Measure similarity differently, based on vagueness of "topic"
ideas:
- use standard deviation in cosine distance to tell something about spread of topic and use this to adjust "similarity"
- Use a measure of deviation that takes into consideration

"std/remove disputed features within category" similarity-algorithm:
- get std per feature, get indexes of the (top 10% or above limit XX) and remove features from topic_vector
- remove features with same index from word to compare similarity with
- do cosine similarity on those features. Have a limit,